##### Multi-Class Classification

###### Dataset link : https://archive.ics.uci.edu/dataset/53/iris

In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [74]:
iris = load_iris()
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['target'] = iris.target

In [75]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [76]:
train_df,test_df = train_test_split(df,test_size=0.2,random_state=42,stratify=df['target'])

In [77]:
X_train,y_train = train_df.drop(columns=['target']),train_df['target']
X_test,y_test = test_df.drop(columns=['target']),test_df['target']


In [78]:
X_train.shape,X_test.shape

((120, 4), (30, 4))

In [79]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [80]:
X_train_tensor = torch.tensor(X_train,dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values,dtype=torch.long)
X_test_tensor = torch.tensor(X_test,dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values,dtype=torch.long)

In [81]:
class IrisClassifier(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super(IrisClassifier,self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,output_dim),
        )

    def forward(self,x):
        return self.network(x)



In [82]:
X_train.shape # 4
input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 3

In [83]:
model = IrisClassifier(input_dim,hidden_dim,output_dim)

In [84]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

###### Model Traning

In [85]:
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    predictions = model(X_train_tensor)
    loss = criterion(predictions,y_train_tensor)
    
    loss.backward()
    optimizer.step()


    if (epoch+1) % 50 == 0:
        print(f"[{epoch+1}/{epochs}],Loss: {loss.item():.4f}") 

[50/500],Loss: 0.0671
[100/500],Loss: 0.0349
[150/500],Loss: 0.0259
[200/500],Loss: 0.0176
[250/500],Loss: 0.0086
[300/500],Loss: 0.0037
[350/500],Loss: 0.0019
[400/500],Loss: 0.0011
[450/500],Loss: 0.0008
[500/500],Loss: 0.0006


In [86]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_labels = torch.argmax(y_pred,dim=1)

    accuracy = (y_pred_labels == y_test_tensor).sum().item() / y_test_tensor.size(0)
    print(f"Test accuracy: {accuracy:.4f}")


Test accuracy: 0.9667


In [87]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [88]:
def predict_iris(sepal_length,sepal_width,petal_length,petal_width):
    input_data = np.array([[sepal_length,sepal_width,petal_length,petal_width]])
    input_data = scaler.transform(input_data)
    input_data_tensor = torch.tensor(input_data,dtype=torch.float32)

    model.eval()
    with torch.no_grad():
        prediction = model(input_data_tensor)
        predicted_class = torch.argmax(prediction,dim=1).item()
    return iris.target_names[predicted_class]



In [91]:
predicted_class  = predict_iris(5.0,3.4,1.5,0.2)
print(f"Predicted Species:{predicted_class}")

Predicted Species:setosa


/home/bhupen/Python_Learning/pytorch-journey/torch-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
